In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [3]:
fake=pd.read_csv("Fake.csv")
true=pd.read_csv("True.csv")


In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class']=0
true['class']=1

In [7]:
data=pd.concat([fake,true],axis=0)

In [8]:
data.sample(10)

,title,text,subject,date,class
1716,"Trump to 'slap' foes, embrace friends in first...",WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"September 15, 2017",1
11045,RUSH LIMBAUGH FURIOUS Trump Is Allowing “Loser...,The Trump administration had proposed $1 billi...,politics,"Apr 26, 2017",0
15738,Zimbabwe accuses American citizen of trying to...,HARARE (Reuters) - Zimbabwean police on Friday...,worldnews,"November 3, 2017",1
13562,BILL O’REILLY RELEASES Never Before Seen Pictu...,Barack Hussein Obama has not exactly done a st...,politics,"Jul 7, 2016",0
10039,Election stirs debate about Fed's handling of ...,WASHINGTON (Reuters) - Donald Trump says the F...,politicsNews,"April 7, 2016",1
15456,Scotland apologizes to gay men for historical ...,EDINBURGH (Reuters) - Scotland s devolved gove...,worldnews,"November 7, 2017",1
6711,"Uber, SpaceX/Tesla, and PepsiCo execs join Tru...","WASHINGTON (Reuters) - Elon Musk, the chairman...",politicsNews,"December 14, 2016",1
10023,Obama defends choice of white male jurist for ...,CHICAGO (Reuters) - President Barack Obama on ...,politicsNews,"April 7, 2016",1
12331,DONALD TRUMP Calls Meeting With Press…Dresses ...,They had to know they had it coming Donald Tru...,politics,"Nov 22, 2016",0
11298,OBAMACARE AND THE FORGOTTEN FAMILY: A Tale Of ...,In light of Republican failure to pass the Ame...,politics,"Mar 25, 2017",0


In [9]:
data = data.drop(["title", "subject","date"], axis = 1)

In [10]:
data.reset_index(inplace=True)

In [11]:
data.drop(['index'],axis=1,inplace=True)

In [12]:
data.sample(5)

,text,class
33945,WASHINGTON (Reuters) - U.S. House of Represent...,1
17469,The video below is one of the highlights from ...,0
14435,"Every time Trump makes lemons from lemonade, t...",0
26646,WASHINGTON (Reuters) - U.S. and Chinese diplom...,1
17942,Maxine Waters got downright nasty at the funer...,0


In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]',"", text)
    text = re.sub("\\W"," " ,text)
    text = re.sub("https?:://\s+\www\.\S+", "", text)
    text = re.sub("<.*?>+","",text)
    text = re.sub("[%s]" % re.escape (string.punctuation), "", text)
    text = re.sub("\n","",text)
    text = re.sub("\w*\d\w*", "", text)
    return text


In [14]:
data["text"] = data["text"].apply(clean_text)

In [15]:
x=data["text"]
y=data["class"]
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [17]:
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

LogisticRegression()

In [18]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [19]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [20]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']